# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# import of Weather csv
vacation_csv = "output_data/cities.csv"
vacation_data = pd.read_csv(vacation_csv)
vacation_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vaini,-21.20,-175.20,75.20,69,75,13.87,TO,1599438878
1,1,zhigansk,66.77,123.37,47.43,75,100,19.04,RU,1599439413
2,2,hithadhoo,-0.60,73.08,82.15,76,100,11.23,MV,1599439413
3,3,cabedelo,-6.98,-34.83,71.60,88,23,3.36,BR,1599439413
4,4,jamestown,42.10,-79.24,68.00,67,1,4.70,US,1599439414


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Store latitude and longitude in locations
locations = vacation_data[["Lat", "Lng"]]

# Fill NaN values in Humidity and convert to float
rating = vacation_data["Humidity"].astype(float)

figure_layout = {
     'width' : '1000px',
     'height':'400px',
     'border':'1px solid black',
     'padding':'1px',
     'margin':'0 auto 0 auto'
 }

fig = gmaps.figure(layout=figure_layout)
#fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=5,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# narrow down cities to maximum temperature between 70 and 80, wind speed less than 5 mph and zero cloudiness.

vacation_area = pd.DataFrame(vacation_data, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness", "Lat", "Lng", "Country"])

max_temp = vacation_area[(vacation_area["Max Temp"] <= 80) & (vacation_area["Max Temp"] > 70)]
wind_speed = max_temp[(max_temp["Wind Speed"] < 10)]
vacation_spot = wind_speed[(wind_speed["Cloudiness"] == 0)]

vacation_spot

,City,Max Temp,Wind Speed,Cloudiness,Lat,Lng,Country
19,kashan,74.50,2.75,0,33.98,51.44,IR
22,mansa,79.74,2.77,0,29.98,75.38,IN
43,arraial do cabo,75.20,8.05,0,-22.84,-46.65,BR
53,nauta,79.32,1.21,0,-4.51,-73.58,PE
56,iquitos,78.80,4.70,0,-3.75,-73.25,PE
83,maku,71.98,6.87,0,39.29,44.45,IR
128,pitimbu,71.60,3.36,0,-7.47,-34.81,BR
131,puerto ayora,75.20,4.70,0,36.59,-6.23,ES
141,kharan,77.43,5.03,0,28.58,65.42,PK
223,waddan,73.54,8.86,0,29.16,16.14,LY


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# add Hotel Name column of Dataframe
hotel_df = vacation_spot.copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Max Temp,Wind Speed,Cloudiness,Lat,Lng,Country,Hotel Name
19,kashan,74.50,2.75,0,33.98,51.44,IR,
22,mansa,79.74,2.77,0,29.98,75.38,IN,
43,arraial do cabo,75.20,8.05,0,-22.84,-46.65,BR,
53,nauta,79.32,1.21,0,-4.51,-73.58,PE,
56,iquitos,78.80,4.70,0,-3.75,-73.25,PE,
83,maku,71.98,6.87,0,39.29,44.45,IR,
128,pitimbu,71.60,3.36,0,-7.47,-34.81,BR,
131,puerto ayora,75.20,4.70,0,36.59,-6.23,ES,
141,kharan,77.43,5.03,0,28.58,65.42,PK,
223,waddan,73.54,8.86,0,29.16,16.14,LY,


In [31]:
# find the first hotels in locations
hotellist = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    hotel_list = response.json()
    try:
        hotellist.append(hotel_list['results'][0]['name'])
    except:
        hotellist.append("")
hotel_df["Hotel Name"] = hotellist
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()
    
       

,City,Country,Lat,Lng,Hotel Name
0,vaini,TO,-21.20,-175.20,
1,zhigansk,RU,66.77,123.37,
2,hithadhoo,MV,-0.60,73.08,
3,cabedelo,BR,-6.98,-34.83,
4,jamestown,US,42.10,-79.24,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
